In [18]:
import time                     # 학습 시간 체크 라이브러리
import random                   # 입실론 그리디를 사용하기 위한 랜덤 라이브러리
import numpy as np              # 넘파이 배열 라이브러리
import torch                    # 토치 딥러닝 라이브러리
import torch.nn as nn           # 토치 모델 생성 라이브러리
import torch.nn.functional as F # 토치 활성화 함수 라이브러리
import torch.optim as optim     # 토치 최적화 함수 라이브러리
import gym                      # gym 강화학습 환경 제공 라이브러리

In [ ]:
class ReplayBuffer():
    # max_size: 버퍼 최대 크기, batch_size: 배치 크기
    def __init__(self, max_size=10000, batch_size=64):
        self.ss_mem = np.empty(shape=(max_size), dtype=np.ndarray) # 상태 리플레이 메모리
        self.as_mem = np.empty(shape=(max_size), dtype=np.ndarray) # 행동 리플레이 메모리
        self.rs_mem = np.empty(shape=(max_size), dtype=np.ndarray) # 보상 리플레이 메모리
        self.ps_mem = np.empty(shape=(max_size), dtype=np.ndarray) # 다음 상태 리플레이 메모리
        self.ds_mem = np.empty(shape=(max_size), dtype=np.ndarray) # 종료 여부 리플레이 메모리

        self.max_size = max_size        # 버퍼 최대 크기
        self.batch_size = batch_size    # 배치 크기
        self._idx = 0                   # 버퍼 인덱스
        self.size = 0                   # 버퍼 현재 사이즈

    def store(self, sample):        # 새로운 리플레이 저장 메소드
        s, a, r, p, d = sample      
        self.ss_mem[self._idx] = s  # 상태 저장
        self.as_mem[self._idx] = a  # 행동 저장
        self.rs_mem[self._idx] = r  # 보상 저장
        self.ps_mem[self._idx] = p  # 다음 상태 저장
        self.ds_mem[self._idx] = d  # 종료 여부 저장

        # 버퍼의 인덱스 저장
        self._idx += 1
        self._idx = self._idx % self.max_size

        # 버퍼의 크기 저장
        self.size += 1
        self.size = min(self.size, self.max_size)

    def sample(self, batch_size=None):      # 배치 크기로 샘플링하는 메소드
        if batch_size == None:              # 배치 크기가 없으면,
            batch_size = self.batch_size    # 인스턴스에서 지정한 배치 크기로 지정
        
        # (버퍼크기~배치크기) 랜덤하게 인덱스값을 불러와 경험 튜플을 반환
        idxs = np.random.choice(self.size, batch_size, replace=False)
        experiences = np.vstack(self.ss_mem[idxs]), np.vstack(self.as_mem[idxs]), np.vstack(self.rs_mem[idxs]), np.vstack(self.ps_mem[idxs]), np.vstack(self.ds_mem[idxs])

        return experiences

    def __len__(self):      # 버퍼의 크기를 반환하는 매직 메서드
        return self.size

In [ ]:
class EGreedyExpStrategy(): # 입실론 그리디 정책을 학습하는 코드
    # init_epsilon: 초기 입실론 값, min_epsilon: 최소 입실론 값, decay_steps: 입실론 감가 횟수
    def __init__(self, init_epsilon=1.0, min_epsilon=0.1, decay_steps=20000):
        self.epsilon = init_epsilon         # 현재 입실론 값
        self.init_epsilon = init_epsilon    # 초기 입실론 값
        self.decay_steps = decay_steps      # 입실론 감가 횟수
        self.min_epsilon = min_epsilon      # 최대 입실론 값

        # 입실론값을 init과 min 사이의 값으로 조정
        self.epsilons = 0.01 / np.logspace(-2, 0, decay_steps, endpoint=False) - 0.01
        self.epsilons = self.epsilons * (init_epsilon - min_epsilon) + min_epsilon

        self.t = 0                              # 현재 입실론 지정 인덱스
        self.exploratory_action_taken = None    # 탐색 여부 저장
    
    def _epsilon_update(self): # 입실론을 선택하고, 인덱스를 증가
        # 감가된 입실론 중 하나 채택
        self.epsilon = self.min_epsilon if self.t >= self.decay_steps else self.epsilons[self.t]
        self.t += 1

        # 채택된 입실론 값 반환
        return self.epsilon
    
    def select_action(self, model, state):      # 감가된 입실론 그리디 전략으로 신경망을 통한 행동 선택
        self.exploratory_action_taken = False   # 탐색이 아님을 표시
        
        with torch.no_grad():   # 연산 기록을 남기지 않아, 속도가 향상
            # 신경망 예측해 Q 함수값을 반환(model(state)), 넘파이로 변환(numpy) 후 미니배치 제거(squeeze)
            q_values = model(state).detach().cpu().data.numpy().squeeze()
        
        if np.random.rand() > self.epsilon:             # 입실론값과 비교해 탐색 여부를 선택
            action = np.argmax(q_values)                # 탐색이 아닌 최대 Q-함수 행동 선택
        else:
            action = np.random.randint(len(q_values))   # 탐색으로 랜덤한 행동 선택
        
        self._epsilon_update()  # 입실론 감가 갱신
        
        # 탐색일 경우 True, 아닐 경우 False
        self.exploratory_action_taken = action != np.argmax(q_values)

        # 선택한 행동 반환
        return action

In [20]:
class DQN():
    def __init__(self, replay_buffer_fn, value_model_fn, value_optimizer_fn,
                 value_optimizer_lr, training_strategy_fn, evaluation_strategy_fn,
                 n_warmup_batches, update_target_every_steps):
        self.replay_buffer_fn = replay_buffer_fn                    # 리플레이 버퍼 람다함수
        self.value_model_fn = value_model_fn                        # 신경망 모델 람다 함수
        self.value_optimizer_fn = value_optimizer_fn                # 신경망 최적화 함수 람다함수
        self.value_optimizer_lr = value_optimizer_lr                # 신경망 최적화 함수 학습률
        self.training_strategy_fn = training_strategy_fn            # 훈련시 행동 선택 전략
        self.evaluation_strategy_fn = evaluation_strategy_fn        # 평가시 행동 선택 전략
        self.n_warmup_batches = n_warmup_batches                    # 샘플링을 위한 최소 샘플 수
        self.update_target_every_steps = update_target_every_steps  # 타겟 신경망 갱신 스텝수

    def optimize_model(self, experiences): # 신경망 학습 모델: 타겟 예측, 손실함수 계산, 역전파 발산
        states, actions, rewards, next_states, is_terminals = experiences           # 경험에 따른 값 반환
        batch_size = len(is_terminals)                                              # 배치 크기 지정
        max_a_q_sp = self.target_model(next_states).detach().max(1)[0].unsqueeze(1) # 신경망 예측 후 Q-함수 계산
        target_q_sa = rewards + (self.gamma * max_a_q_sp * (1 - is_terminals))      # 타겟값 계산
        q_sa = self.online_model(states).gather(1, actions)                         # 현재 상태 Q-함수 예측
        td_error = q_sa - target_q_sa                                               # 오차 계산
        value_loss = td_error.pow(2).mul(0.5).mean()                                # 손실함수 계산
        self.value_optimizer.zero_grad()                                            # 최적화 함수의 기울기 초기화
        value_loss.backward()                                                       # 오차를 바탕으로 역전파 발산
        self.value_optimizer.step()                                                 # 최적화 함수를 통한 갱신

    def interaction_step(self, state, env): # 입실론 그리디 전략으로 행동 후 다음 상태 전이
        action = self.training_strategy.select_action(self.online_model, state) # 행동 선택
        new_state, reward, terminated, truncated, info = env.step(action)       # 행동 수행 후 상태 전이
        done = terminated or truncated                                          # 종료 여부 계산
        experience = (state, action, reward, new_state, float(done))            # 경험 튜플 저장
        self.replay_buffer.store(experience)                                    # 리플레이 버퍼에 경험 튜플 저장
        self.episode_reward[-1] += reward                                       # 보상 저장
        self.episode_timestep[-1] += 1                                          # 타임 스텝 저장

        # 탐색 횟수 증가
        self.episode_exploration[-1] += int(self.training_strategy.exploratory_action_taken)

        # 새로운 상태와 종료 여부 반환
        return new_state, done
    
    def update_network(self): # 온라인 신경망 파라미터로 타겟 신경망 갱신
        for target, online in zip(self.target_model.parameters(), self.online_model.parameters()):
            target.data.copy_(online.data)

    # 신경망 훈련 메소드
    def train(self, env, seed, gamma, max_minutes, max_episodes, goal_mean_100_reward):
        # 신경망 훈련 시작 시간을 저장
        training_start, last_debug_time = time.time(), float("-inf")

        self.seed = seed                # 시드 지정
        self.gamma = gamma              # 할인율 지정
        torch.manual_seed(self.seed)    # 랜덤 시드 지정
        np.random.seed(self.seed)       # 랜덤 시드 지정
        random.seed(self.seed)          # 랜덤 시드 지정

        # 상태 개수, 행동 수 저장
        nS, nA = env.observation_space.shape[0], env.action_space.n

        self.episode_timestep = []      # 에피소드 타임 스텝 리스트
        self.episode_reward = []        # 에피소드 보상 리스트
        self.episode_seconds = []       # 에피소드 초 리스트
        self.evaluation_scores = []     # 에피소드 점수 리스트
        self.episode_exploration = []   # 에피소드 경험 리스트

        self.target_model = self.value_model_fn(nS, nA) # 타겟 신경망 객체 생성
        self.online_model = self.value_model_fn(nS, nA) # 온라인 신경망 객체 생성
        self.update_network()                           # 타겟과 온라인 신경망 초기화

        self.value_optimizer = self.value_optimizer_fn(self.online_model, self.value_optimizer_lr)  # 최적화 함수 지정
        self.replay_buffer = self.replay_buffer_fn()                                                # 리플레이 버퍼 지정
        self.training_strategy = self.training_strategy_fn()                                        # 훈련 전략 지정
        self.evaluation_strategy = self.evaluation_strategy_fn()                                    # 검증 전략 지정

        result = np.empty((max_episodes, 5))    # 에피소드 및 5개의 정보를 저장하기 위한 공간 생성
        result[:] = np.nan                      # 결과 값을 모두 초기화
        training_time = 0                       # 훈련 시간 초기화

        # 에피소드 횟수만큼 반복, 변수를 쓰기 위해 +1 한 후 사용
        for episode in range(1, max_episodes + 1):
            episode_start = time.time()                     # 에피소드 시작 시간 저장
            state, done = env.reset(seed=self.seed), False  # 환경 초기화
            state = state[0]                                # 상태 변수 저장
            self.episode_reward.append(0.0)                 # 에피소드 보상값 초기화 및 공간 할당
            self.episode_timestep.append(0.0)               # 에피소드 타임스텝 초기화 및 공간 할당
            self.episode_exploration.append(0.0)            # 에피소드 탐색여부 초기화 및 공간 할당

            # 하나의 에피소드 종료시까지 반복
            while not done:
                state, done = self.interaction_step(state, env)                     # 행동 수행 후 상태 전이
                min_samples = self.replay_buffer.batch_size * self.n_warmup_batches # 샘플링을 위한 최소 버퍼 크기 지정

                if len(self.replay_buffer) > min_samples:               # 미니배치 만큼 샘플링
                    experiences = self.replay_buffer.sample()           # 리플레이 버퍼에서 샘플링
                    experiences = self.online_model.load(experiences)   # 경험 튜플을 텐서로 변환해 저장
                    self.optimize_model(experiences)                    # 신경망 최적화 실행

                # 타임스텝 10마다 타겟 신경망 갱신
                if np.sum(self.episode_timestep) % self.update_target_every_steps == 0:
                    self.update_network()
            
            episode_elapsed = time.time() - episode_start   # 경과시간 계산
            self.episode_seconds.append(episode_elapsed)    # 경과시간 리스트에 추가
            training_time += episode_elapsed                # 훈련 시간 계산
            total_step = int(np.sum(self.episode_timestep)) # 누적 타임 스텝 계산

            # 훈련 종료 후 모델 평가 수행 및 그래프 값 계산
            evaluation_score, _ = self.evaluate(self.online_model, env)
            self.evaluation_scores.append(evaluation_score)
            mean_10_reward = np.mean(self.episode_reward[-10:])
            std_10_reward = np.std(self.episode_reward[-10:])
            mean_100_reward = np.mean(self.episode_reward[-100:])
            std_100_reward = np.std(self.episode_reward[-100:])
            mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
            std_100_eval_score = np.std(self.evaluation_scores[-100:])
            lst_100_exp_rat = np.array(self.episode_exploration[-100:]) / np.array(self.episode_timestep[-100:])
            mean_100_exp_rat = np.mean(lst_100_exp_rat)
            std_100_exp_rat = np.std(lst_100_exp_rat)
            wallclock_elapsed = time.time() - training_start
            result[episode-1] = total_step, mean_100_reward, mean_100_eval_score, mean_100_exp_rat, wallclock_elapsed
            LEAVE_PRINT_EVERY_N_SECS = 60
            ERASE_LINE = "\x1b[2K"
            reached_debug_time = time.time() - last_debug_time >= LEAVE_PRINT_EVERY_N_SECS
            reached_max_minutes = wallclock_elapsed >= max_minutes * 60
            reached_max_episodes = episode >= max_episodes
            reached_goal_mean_reward = mean_100_reward >= goal_mean_100_reward
            training_is_over = reached_max_minutes or reached_max_episodes or reached_goal_mean_reward
            elapsed_str = time.strftime("%H:%M:%S", time.gmtime(time.time() - training_start))
            debug_message = 'el {}, ep {:04}, ts {:06}, '
            debug_message += 'ar 10 {:05.1f}\u00B1{:05.1f}, '
            debug_message += '100 {:05.1f}\u00B1{:05.1f}, '
            debug_message += 'ex 100 {:02.1f}\u00B1{:02.1f}, '
            debug_message += 'ev {:05.1f}\u00B1{:05.1f}'
            debug_message = debug_message.format(
                elapsed_str, episode-1, total_step,
                mean_10_reward, std_10_reward,
                mean_100_reward, std_100_reward,
                mean_100_exp_rat, std_100_exp_rat,
                mean_100_eval_score, std_100_eval_score)
            print(debug_message, end='\r', flush=True)

            if reached_debug_time or training_is_over:
                print(ERASE_LINE + debug_message, flush=True)
                last_debug_time = time.time()
            
            if training_is_over:
                if reached_max_minutes: print(u"--> reached_max_minutes \u2715")
                if reached_max_episodes: print(u"--> reached_max_episodes \u2715")
                if reached_goal_mean_reward: print(u"--> reached_goal_mean_reward \u2713")
                break
        
        final_eval_score, score_std = self.evaluate(self.online_model, env, n_episodes=100)

        wallclock_time = time.time() - training_start
        print("Training complete.")
        print(f"Final evaluation score {final_eval_score:.2f}\u00B1{score_std:.2f} in {training_time:.2f}s training time, {wallclock_time:.2f}s wall-clock time.\n")
        env.close()
        del env

        return result, final_eval_score, training_time, wallclock_time
    
    # 에피소드 및 훈련 종료 후 그리디 전략으로 보상을 계산하는 메소드
    def evaluate(self, eval_policy_model, eval_env, n_episodes=1):
        rs = [] # 보상 리스트 초기화

        for _ in range(n_episodes):             # 에피소드 만큼 반복
            s, done = eval_env.reset(), False   # 환경 초기화
            s = s[0]                            # 상태 저장
            rs.append(0)                        # 보상 리스트에 공간 할당

            # 에피소드 종료시까지 반복
            while not done:
                a = self.evaluation_strategy.select_action(eval_policy_model, s) # 행동을 선택
                s, r, terminated, truncated, info = eval_env.step(a)             # 하나의 스텝 진행
                done = terminated or truncated                                   # 종료 여부 판단
                rs[-1] += r                                                      # 리스트 끝에 계산된 보상 저장
    
    # 학습 후 그리디 전략으로 렌더링 해주는 메소드
    def render_after_train(self, r_env, n_episodes=1):
        for _ in range(n_episodes):         # 에피소드 횟수만큼 반복
            s, done = r_env.reset(), False  # 환경 초기화
            s = s[0]                        # 상태 저장

            # 에피소드 종료시까지 반복
            while not done:
                a = self.evaluation_strategy.select_action(self.online_model, s)    # 행동 선택
                s, r, terminated, truncated, info = r_env.step(a)                   # 스텝 진행
                done = terminated or truncated                                      # 종료 여부 반환

In [22]:
class FCQ(torch.nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims=(32, 32), activation_fc=torch.nn.functional.relu):
        super(FCQ, self).__init__()         # 기본 클래스의 매개변수들을 그대로 사용합니다.
        self.activation_fc = activation_fc  # 활성화 함수, 기본값은 relu

        self.input_layer = torch.nn.Linear(input_dim, hidden_dims[0])   # 입력 레이어입니다.
        self.hidden_layers = torch.nn.ModuleList()  # 동적인 은닉층을 생성할 때, 리스트에 넣어 사용합니다.

        for i in range(len(hidden_dims)-1): # 은닉층 차원수만큼 생성합니다.
            hidden_layer = torch.nn.Linear(hidden_dims[i], hidden_dims[i+1]) # 은닉층 완전연결계층을 생성합니다.
            self.hidden_layers.append(hidden_layer) # 생성된 완전연결계층을 ModuleList에 추가합니다.
        
        self.output_layer = torch.nn.Linear(hidden_dims[-1], output_dim) # 출력 레이어입니다.

        device = "cpu"                  # 기본 device cpu로 설정
        if torch.cuda.is_available():   # torch의 cuda 설정 가능할 경우
            device = "cuda:0"           # device cuda로 설정
        
        self.device = torch.device(device)  # 설정된 device로 학습
        self.to(self.device)                # 모듈의 파라미터/버퍼 등을 해당 디바이스로 이동 (casting)

    def _format(self, state):   # 입력값이 텐서가 아니면 텐서로 변환
        x = state

        if not isinstance(x, torch.Tensor): # 입력값의 인스턴스가 파이토치 텐서인지 비교
            x = torch.tensor(x, device=self.device, dtype=torch.float32)    # 아닐 경우 토치형식으로 새로 생성
            x = x.unsqueeze(0)
        
        return x
    
    def forward(self, state):   # 신경망 순전파 함수
        x = self._format(state)                     # 토치 텐서로 변환
        # x = self.activate_fc(self.input_layer(x))   # 활성화 함수 설정
        x = self.activation_fc(self.input_layer(x))   # 코드 수정

        for hidden_layer in self.hidden_layers:     # 은닉층의 수에 따라 반복
            # x = self.activate_fc(hidden_layer(x))   # 활성화 함수 지정
            x = self.activation_fc(hidden_layer(x))   # 코드 수정
        
        x = self.output_layer(x)                    # 출력 레이어 설정

        return x
    
    def load(self, experiences): # 경험 튜플을 텐서로 변환해주는 메서드
        states, actions, new_states, rewards, is_terminals = experiences

        states = torch.from_numpy(states).float().to(self.device)               # 상태
        actions = torch.from_numpy(actions).long().to(self.device)              # 행동
        new_states = torch.from_numpy(new_states).float().to(self.device)       # 다음 상태
        rewards = torch.from_numpy(rewards).float().to(self.device)             # 보상
        is_terminals = torch.from_numpy(is_terminals).float().to(self.device)   # 종료 여부

        return states, actions, new_states, rewards, is_terminals

In [23]:
class EGreedyStrategy(): # 입실론 그리디 탐색 전략으로 훈련을 진행합니다.
    def __init__(self, epsilon=0.1): # 기본 입실론 값은 0.1으로 지정
        self.epsilon = epsilon
        self.exploratory_action_taken = None # 선택한 행동이 탐색인지에 대한 여부 확인

    def select_action(self, model, state):
        self.exploratory_action_taken = False

        with torch.no_grad(): # 연산 기록을 하지 않아, 검증 속도가 향상됩니다.
            q_values = model(state).cpu().detach().data.numpy().squeeze()
        
        if np.random.rand() > self.epsilon: # 입실론 값보다 크면
            action = np.argmax(q_values)    # 최대 보상을 가진 행동으로 선택합니다.
        else:                                           # 입실론 값보다 작으면
            action = np.random.randint(len(q_values))   # 랜덤의 행동을 선택합니다.
        
        self.exploratory_action_taken = action != np.argmax(q_values) # 탐색 여부 저장

        return action

In [24]:
class GreedyStrategy():
    def __init__(self):
        self.exploratory_action_taken = False # 탐색인지 확인
    
    def select_action(self, model, state): # 행동을 선택하는 메서드
        with torch.no_grad(): # 기울기를 저장하지 않아, 검증 속도가 향상됩니다.
            q_values = model(state).cpu().detach().data.numpy().squeeze()
        
        return np.argmax(q_values)

In [28]:
dqn_results = []    # 결과 리스트 초기화

SEEDS = (12, 34, 56, 78, 90)    # 5개의 시드에 대해 학습 진행

# 5개의 시드에 대해 학습 진행
for seed in SEEDS:
    environment_settings = {
        "env_name" : "CartPole-v1",     # CartPole 환경으로 초기화
        "gamma" : 1.00,                 # 할인율 지정
        "max_minutes" : 20,             # 최대 학습 시간 지정
        "max_episodes" : 10000,         # 최대 에피소드 수 지정
        "goal_mean_100_reward" : 475    # 목표 보상 지정
    }

    # 신경망 모델 람다 함수
    value_model_fn = lambda nS, nA: FCQ(nS, nA, hidden_dims=(512, 128))

    # 신경망 최적화 람다 함수, 신경망 최적화 함수 학습률
    value_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)
    value_optimizer_lr = 0.0005

    # 훈련 입실론 그리디 전략 람다 함수
    training_strategy_fn = lambda: EGreedyExpStrategy(init_epsilon=1.0, min_epsilon=0.3, decay_steps=20000)

    # 평가 입실론 그리디 전략 람다 함수
    evaluation_strategy_fn = lambda: GreedyStrategy()

    # 리플레이 버퍼 지정 람다 함수
    replay_bufferr_fn = lambda: ReplayBuffer(max_size=50000, batch_size=64)
    n_warmup_batches = 5            # 워밍업 배치 크기
    update_target_every_steps = 10  # 타겟 신경망 갱신 스텝수

    # 튜플에서 값 가져오기
    env_name, gamma, max_minutes, max_episodes, goal_mean_100_reward = environment_settings.values()

    env = gym.make(env_name)    # 환경 지정
    # DQN 에이전트 생성
    agent = DQN(replay_bufferr_fn, value_model_fn, value_optimizer_fn, value_optimizer_lr, training_strategy_fn, evaluation_strategy_fn, n_warmup_batches, update_target_every_steps)

    # DQN 에이전트 학습
    result, final_eval_score, training_time, wallclock_time = agent.train(env, seed, gamma, max_minutes, max_episodes, goal_mean_100_reward)

    dqn_results.append(result)  # 결과를 리스트에 저장

dqn_results = np.array(dqn_results) # 넘파이 배열로 전환
agent.render_after_train(env)       # 학습 후 카트폴 동작 렌더링

# 그래프 출력용 변수 선언
dqn_max_t, dqn_max_r, dqn_max_s, dqn_max_sec, dqn_max_rt = np.max(dqn_results, axis=0).T
dqn_min_t, dqn_min_r, dqn_min_s, dqn_min_sec, dqn_min_rt = np.min(dqn_results, axis=0).T
dqn_mean_t, dqn_mean_r, dqn_mean_s, dqn_mean_sec, dqn_mean_rt = np.mean(dqn_results, axis=0).T
dqn_x = 2200

el 00:00:00, ep 0000, ts 000017, ar 10 017.0±000.0, 100 017.0±000.0, ex 100 0.4±0.0, ev 026.0±000.0


c:\Users\dldls\anaconda3\envs\study3-2\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

plt.style.use('fivethirtyeight')
params = {
    "figure.figsize": (15, 8), "font.size": 24, "legend.fontsize": 20,
    "axes.titlesize": 28, "axes.labelsize": 24, "xtick.labelsize": 20, "ytick.labelsize": 20
}

pylab.rcParams.update(params)

fig, axs = plt.subplots(5, 1, figsize=(15, 30), sharey=False, sharex=True)

axs[0].plot(dqn_max_r, "y", linewidth=1)
axs[0].plot(dqn_min_r, "y", linewidth=1)
axs[0].plot(dqn_mean_r, "y", label="dqn", linewidth=2)
axs[0].fill_between(dqn_x, dqn_min_r, dqn_max_r, facecolor="y", alpha=0.3)

axs[1].plot(dqn_max_s, "y", linewidth=1)
axs[1].plot(dqn_min_s, "y", linewidth=1)
axs[1].plot(dqn_mean_s, "y", label="dqn", linewidth=2)
axs[1].fill_between(dqn_x, dqn_min_s, dqn_max_s, facecolor="y", alpha=0.3)

axs[2].plot(dqn_max_t, "y", linewidth=1)
axs[2].plot(dqn_min_t, "y", linewidth=1)
axs[2].plot(dqn_mean_t, "y", label="dqn", linewidth=2)
axs[2].fill_between(dqn_x, dqn_min_t, dqn_max_t, facecolor="y", alpha=0.3)

axs[3].plot(dqn_max_sec, "y", linewidth=1)
axs[3].plot(dqn_min_sec, "y", linewidth=1)
axs[3].plot(dqn_mean_sec, "y", label="dqn", linewidth=2)
axs[3].fill_between(dqn_x, dqn_min_sec, dqn_max_sec, facecolor="y", alpha=0.3)

axs[4].plot(dqn_max_rt, "y", linewidth=1)
axs[4].plot(dqn_min_rt, "y", linewidth=1)
axs[4].plot(dqn_mean_rt, "y", label="dqn", linewidth=2)
axs[4].fill_between(dqn_x, dqn_min_rt, dqn_max_rt, facecolor="y", alpha=0.3)

axs[0].set_title("dqn: Moving Avg Reward (Training)")
axs[1].set_title("dqn: Moving Avg Reward (Evaluation)")
axs[2].set_title("dqn: Total Steps")
axs[3].set_title("dqn: Training Time")
axs[4].set_title("dqn: Wall-clock Time")

plt.xlabel("Episodes")
axs[0].legend(loc="upper left")
plt.show()